## 메모리 기능이 추가된 금융 고객센터 챗봇

이 Practice는 금융 고객센터 상담 시나리오를 다룹니다. 고객이 자신의 금융 개인정보(예: 이름, 계좌번호 등)를 입력하고, `remember` 라는 키워드로 "기억"을 요청합니다. 이후 다른 상담 스레드(다른 thread_id)에서 동일한 고객(user_id)이 다시 질문하면, 이전에 저장된 메모리를 바탕으로 불필요한 재질문 없이 자연스럽게 응대하는 챗봇을 구현합니다.

### 목표
- 메시지 상태(`MessagesState`)를 사용하는 LangGraph 노드를 구성합니다.
- 사용자별 메모리를 저장/검색하기 위한 Store와 Checkpointer를 구성합니다.
- `remember` 키워드가 포함된 입력을 자동 추출하여 메모리에 저장합니다.
- 서로 다른 thread에서 동일한 user_id로 상담 시, 저장된 메모리를 활용해 응답 품질을 높입니다.

### 조건
- LLM 모델: `gpt-4.1` (온도 0)
- 메모리 저장: `user_id` 기반 네임스페이스로 구분
- 키 포인트: `remember` 가 포함된 입력이면 메모리 추출 → 저장, 이후 대화에 반영

---

아래 각 소파트에는
- 실습 코드: 스스로 채워 넣는 TODO 형태
- 정답 코드: 참고용 구현
이 함께 제공됩니다. 우선 실습 코드를 시도해 본 뒤, 정답 코드를 참고하세요.


In [ ]:
# 공통 준비 코드 (실행)
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)
logging.langsmith("LangGraph-Memory-Finance-CC-Practice")

### 1) LLM과 메모리 추출기 준비

금융 고객센터 시나리오에 사용할 LLM과 간단한 메모리 추출기를 준비합니다. 추출기는 `remember` 키워드가 포함된 메시지에서 개인정보(예: 이름, 계좌번호, 생년월일 등)를 구조화해 저장하기 위함입니다.

요구사항
- `ChatOpenAI`로 `gpt-4.1`, `temperature=0` 모델을 생성하세요.
- `create_memory_extractor(model="gpt-4.1")` 와 유사한 인터페이스의 추출기를 준비한다고 가정하고, 변수를 선언하세요.


In [ ]:
# 실습 코드
# TODO: LLM과 메모리 추출기 변수를 준비하세요.
from langchain_openai import ChatOpenAI
from langchain_teddynote.memory import create_memory_extractor

llm = None  # 코드 입력
memory_extractor = None  # 코드 입력

### 2) 상태와 그래프 골격 준비

메시지 상태는 LangGraph의 `MessagesState`를 사용합니다. 고객의 메시지와 상담사의 답변이 누적되도록 구성하고, 나중에 체크포인팅과 Store를 연결합니다.

요구사항
- `MessagesState` 기반 `StateGraph` 빌더를 생성하세요.
- START → `call_model` 단일 노드로 시작하는 최소 골격을 준비하세요.


In [ ]:
# 실습 코드
# TODO: MessagesState 기반 그래프 빌더를 생성하고 START→call_model 골격을 만드세요.
from typing import Any
from langgraph.graph import StateGraph, MessagesState, START

builder = None  # 코드 입력

# 노드는 다음 소파트에서 구현 후 추가합니다.
# 최소 골격: START → call_model
# (노드 추가는 이후 단계에서 실제 함수 구현 후)

### 3) 메모리 로드/저장 로직이 포함된 노드 구현

`call_model(state, config, *, store)` 형태로 노드를 구현합니다.
- 최근 사용자 메시지에서 `user_id` 를 `config["configurable"]["user_id"]` 로 가져옵니다.
- 네임스페이스: `("memories", user_id)` 로 지정합니다.
- 마지막 메시지에 `remember` 가 포함되어 있으면 `memory_extractor` 를 이용해 개인정보를 추출하고 Store에 저장합니다.
- 저장된 메모리를 조회해 system 프롬프트(컨텍스트)로 반영한 후 LLM을 호출합니다.

요구사항
- 함수 시그니처와 반환 스키마를 맞추세요: `{"messages": response}` 또는 `{"messages": [response]}` (여기서는 하나의 AI 메시지를 반환)
- Store 검색 결과를 system 메시지로 앞단에 추가해 답변 품질 향상


In [ ]:
# 실습 코드
# TODO: 메모리 로드/저장 로직이 포함된 노드를 작성하세요.
from typing import Any
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore


def call_model(state, config: RunnableConfig, *, store: BaseStore) -> dict[str, Any]:
    # 1) user_id와 namespace 추출
    # 2) 마지막 메시지에 remember 포함 여부 확인 → 포함 시 memory_extractor로 추출 후 store.put
    # 3) store.search로 저장된 메모리 검색 → system 메시지 구성
    # 4) LLM 호출 후 메시지 반환
    raise NotImplementedError  # 코드 입력

### 4) Checkpointer/Store 연결 및 그래프 컴파일

메모리를 유지하려면 체크포인터와 스토어가 필요합니다. 여기서는 메모리 기반 구현을 사용하지만, 실전에서는 외부 DB(PostgresSaver 등)를 권장합니다.

요구사항
- `InMemorySaver` 와 `InMemoryStore` 를 생성하여 그래프 컴파일 시 주입하세요.
- 앞서 만든 `call_model` 노드를 그래프에 추가하고, `START → call_model` 엣지를 구성하세요.
- 컴파일된 그래프를 `graph_with_memory` 라는 변수에 담으세요.


In [ ]:
# 실습 코드
# TODO: InMemorySaver/Store로 그래프 컴파일 및 노드/엣지 구성
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

memory_saver = # 코드 입력
memory_store = # 코드 입력

builder. # 코드 입력
builder. # 코드 입력

graph_with_memory = # 코드 입력

### 5) 시나리오 실행 1: 개인정보 등록 + remember

아래 예시는 최초 상담에서 고객이 자신의 정보를 제공하며 `remember` 를 명시하는 경우입니다. 동일한 `user_id` 와 `thread_id` 를 전달하세요.

요구사항
- `RunnableConfig` 의 `configurable`에 `thread_id`, `user_id`를 설정하세요.
- "제 이름은 홍길동이고, 계좌번호는 110-123-456789 입니다. remember" 와 같은 문장을 입력하여 메모리 저장을 유도하세요.
- `stream_graph` 로 실행 결과를 확인하세요.


In [ ]:
# 실습 코드
# TODO: 동일 thread_id/user_id로 개인정보 + remember 시나리오 실행
from langchain_teddynote.messages import stream_graph
from langchain_core.runnables import RunnableConfig

config1 = RunnableConfig(
    configurable={"thread_id": "t-001user-abc", "user_id": "user-abc"}
)
inputs1 = {
    "messages": [
        {
            "role": "user",
            "content": "제 이름은 홍길동이고, 계좌번호는 110-123-456789 입니다. remember",
        }
    ]
}

# 코드 입력 (stream_graph 로 실행)

### 6) 시나리오 실행 2: 다른 thread에서 동일 user로 재상담

이제 thread만 바꿔 동일한 `user_id`로 다시 질문합니다. 챗봇은 기존 메모리를 바탕으로 불필요한 개인정보 재질문 없이 바로 답변해야 합니다.

요구사항
- `thread_id`만 변경하고, `user_id`는 동일하게 유지하세요.
- "어제 안내해준 대출 상환 스케줄 다시 알려줘" 같은 문장을 사용해 답변을 확인하세요.


In [ ]:
# 실습 코드
# TODO: 다른 thread_id, 같은 user_id로 실행하여 메모리 활용 확인
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph

config2 = RunnableConfig(
    configurable={"thread_id": "t-002user-abc", "user_id": "user-abc"}
)
inputs2 = {"messages": [{"role": "user", "content": "제 정보를 알려주세요"}]}

stream_graph(graph_with_memory, inputs=inputs2, config=config2)